In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo 
import pandas as pd

In [29]:
#Importing Iris dataset as Pandas dataframe.
iris_dataset = fetch_ucirepo(id=53) 

iris_X = iris_dataset.data.features 
iris_Y = iris_dataset.data.targets 

class_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

iris_Y['class'] = iris_Y['class'].map(class_mapping)

print(iris_Y)

     class
0        0
1        0
2        0
3        0
4        0
..     ...
145      2
146      2
147      2
148      2
149      2

[150 rows x 1 columns]


C:\Users\Shiyan Liu\AppData\Local\Temp\ipykernel_4600\3920779809.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iris_Y['class'] = iris_Y['class'].map(class_mapping)


In [30]:
class BinningLayer(torch.nn.Module):
    def __init__(self, num_cutting_points, tau, feature_min, feature_max):
        super().__init__()
        
        #Constant binner in the first softmax layer. Not trained
        self.w=torch.nn.Parameter(torch.Tensor([i+1 for i in range(num_cutting_points+1)]), requires_grad=False)
        
        #Cut points, randomized between the minimum and maximum values for each feature.c
        cut_points=(feature_min + (feature_max - feature_min) * torch.rand(n)).numpy()
        self.b = nn.Parameter(torch.Tensor([0]+[-cut_points[i] for i in range(cut_points)]))
        for i in range(1, self.b):
            self.b[i]+=self.b[i-1]

        self.tau=tau #Temperature parameter for the first softmax layer.
        
    def forward(self, x):
        x=torch.add(torch.multiply(self.w, x), b)
        x=torch.nn.Softmax(torch.div(x, self.tau))
        return x
    
class DNDT(torch.nn.Module):
    def __init__(self, X_frame, Y_frame, num_cutting_points=1, tau=0.01):
        super().__init__()
        self.X=X_frame #Features dataframe in pandas.
        self.Y=Y_frame #Targets dataframe in pandas.
        self.num_cutting_points=num_cutting_points #Number of cutting points for each feature.
        self.tau=tau
        
        self.range_matrix=self.create_range_matrix() #Dx2 matrix, where each length 2 matrix is the min and
            #max of each feature in your dataset.
        self.binning_layers=self.create_binning_layers() #D matrix, where the i-entry is a NN model for
            #soft binning the i-th feature.
        
        self.num_features=self.X.shape[1] #D
        self.num_classes=max(self.Y) #C
        self.linear_layer=torch.nn.Linear((num_cutting_points+1)**(self.num_features), self.numClasses) #Linear 
            #classifier layer.
        
    #Creates binning layers list.
    def create_binning_layers:
        res=[]*self.X.shape[1]
        for i in range(len(res)):
            res[i]=BinningLayer(num_cutting_points, self.tau, self.range_matrix[i][0], self.range_matrix[i][1])
        return res
    
    #Creates range matrix list.
    def create_range_matrix(self):
        res=[[0,0] for i in range(self.X.shape[1])]
        columns=self.X.columns
        i=0
        for column in columns:
            res[i][0]=min(self.X[column])
            res[i][1]=max(self.X[column])
            i+=1
        return res
            
    def kronecker_prod(self, x):
        result=x[0]
        for i in range(1, len(x)):
            result=torch.kron(result, x[i])
        return result
    
    def forward(self, x):
        x=torch.stack([self.binning_layers[i](x[i]) for i in range(self.num_features)])
        x=self.kronecker_prod(x)
        x=self.linear_layer(x, bias=True)
        x=torch.nn.Softmax(x)
        return x
        
        

SyntaxError: expected '(' (3122132769.py, line 38)

In [27]:
deep_decision_tree=DNDT(iris_X, iris_Y, 1)


[[4.3, 7.9], [2.0, 4.4], [1.0, 6.9], [0.1, 2.5]]
